<h2 style='color:purple' align='center'>Naive Bayes: Predicting Survival from Titanic Crash</h2>

In [24]:
import pandas as pd

In [25]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [26]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [27]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [28]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [29]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [30]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [31]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [32]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [33]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [45]:
inputs.Age = inputs.Age.fillna(inputs.Age.median())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [36]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [37]:
model.fit(X_train,y_train)

GaussianNB()

In [38]:
model.score(X_test,y_test)

0.7798507462686567

In [39]:
X_test[0:10]

,Pclass,Age,Fare,female
618,2,4.0,39.0000,1
834,3,18.0,8.3000,0
349,3,42.0,8.6625,0
707,1,42.0,26.2875,0
623,3,21.0,7.8542,0
86,3,16.0,34.3750,0
3,1,35.0,53.1000,1
818,3,43.0,6.4500,0
753,3,23.0,7.8958,0
299,1,50.0,247.5208,1


In [40]:
y_test[0:10]

618    1
834    0
349    0
707    1
623    0
86     0
3      1
818    0
753    0
299    1
Name: Survived, dtype: int64

In [41]:
model.predict(X_test[0:10])

array([1, 0, 0, 0, 0, 0, 1, 0, 0, 1], dtype=int64)

In [42]:
model.predict_proba(X_test[:10])

array([[1.82002387e-01, 8.17997613e-01],
       [9.63512716e-01, 3.64872836e-02],
       [9.68935433e-01, 3.10645674e-02],
       [7.35396833e-01, 2.64603167e-01],
       [9.65022357e-01, 3.49776430e-02],
       [9.56214664e-01, 4.37853365e-02],
       [5.35310649e-02, 9.46468935e-01],
       [9.68462233e-01, 3.15377669e-02],
       [9.65919501e-01, 3.40804993e-02],
       [1.31206011e-11, 1.00000000e+00]])

**Calculate the score using cross validation**

In [43]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.736     , 0.76      , 0.768     , 0.77419355, 0.73387097])